In [1]:
from resets import *
from prefutils import *

# initialize web cache
SESSION = init_fetch_session()

## Setup global for now
test_month_cycle = [3, 6, 9, 12]


        

In [2]:
# Next Steps
#
# 1. read in CSV table of fixed resets; update prices
# 2. compute the current market spread
# 3. compute future dividend (add to table)
# 4. given a market spread, compute price as a function of various GOC5 estimate
# 5. loop over multiple MS scenarios. Unlike specifying explicitly, use functional guess


resets = pd.read_csv('./resets.csv', dtype={'Notes': object})

# convert dates
resets['ResetDT'] = [csv_to_date(x) for x in resets['Reset Date']]

resets.drop(columns=['Notes','RefPrice','Reset Date'],inplace=True,errors='ignore')

df = resets.copy()
TICKER_BLACKLIST = ['BBD.PR.D', 'GMP.PR.B', 'BIR.PR.A','CSE.PR.A']

# remove unacceptable tickers
df = df[[x not in TICKER_BLACKLIST for x in df['Ticker']]]

RATING_BLACKLIST = ['P3Im', 'P2Hb', 'ZRm', 'P6', 'P2Ln','P2Lm']

# remove unreasonable ratings for now
df = df[[x not in RATING_BLACKLIST for x in df['Rating']]]
df
# Convert cycle code to actual datetimes
df["Cycle"] = [month_cycle_from_start_month(x) for x in df["CycleCode"]]
df.drop(columns=['CycleCode'],inplace=True,errors='ignore')

In [3]:

df

,Ticker,Rating,Div,Spread,ResetDT,Cycle
0,AIM.PR.A,ZR,1.12500,375,2025-03-15,"[3, 6, 9, 12]"
1,AIM.PR.C,ZR,1.50250,420,2024-03-15,"[3, 6, 9, 12]"
2,ALA.PR.A,P3I,0.84500,266,2020-09-15,"[3, 6, 9, 12]"
3,ALA.PR.E,P3I,1.25000,317,2023-12-15,"[3, 6, 9, 12]"
4,ALA.PR.G,P3I,1.06050,306,2024-09-15,"[3, 6, 9, 12]"
...,...,...,...,...,...,...
165,TRP.PR.B,P2L,0.53800,128,2020-06-15,"[2, 5, 8, 11]"
166,TRP.PR.C,P2L,0.56575,154,2021-01-15,"[2, 5, 8, 11]"
167,TRP.PR.D,P2L,1.00000,238,2024-04-15,"[2, 5, 8, 11]"
168,TRP.PR.E,P2L,0.94050,235,2024-10-15,"[2, 5, 8, 11]"


In [4]:
df = update_data_frame_with_prices_and_drop_reference(df, SESSION)
df

,Ticker,Rating,Div,Spread,ResetDT,Cycle,Price
0,AIM.PR.A,ZR,1.12500,375,2025-03-15,"[3, 6, 9, 12]",15.51
1,AIM.PR.C,ZR,1.50250,420,2024-03-15,"[3, 6, 9, 12]",19.08
2,ALA.PR.A,P3I,0.84500,266,2020-09-15,"[3, 6, 9, 12]",13.30
3,ALA.PR.E,P3I,1.25000,317,2023-12-15,"[3, 6, 9, 12]",18.15
4,ALA.PR.G,P3I,1.06050,306,2024-09-15,"[3, 6, 9, 12]",16.25
...,...,...,...,...,...,...,...
165,TRP.PR.B,P2L,0.53800,128,2020-06-15,"[2, 5, 8, 11]",9.81
166,TRP.PR.C,P2L,0.56575,154,2021-01-15,"[2, 5, 8, 11]",10.45
167,TRP.PR.D,P2L,1.00000,238,2024-04-15,"[2, 5, 8, 11]",15.23
168,TRP.PR.E,P2L,0.94050,235,2024-10-15,"[2, 5, 8, 11]",14.99


In [5]:
CURRENT_GOC5_PERCENT = 0.88

foo = update_dataframe_with_market_spread_from_dividend(df, CURRENT_GOC5_PERCENT)
foo = update_dataframe_with_rating_averages(foo)



In [59]:
GOC5_SCN1 = { 
    "Constant" :  (0.74,  0.50),
   "SlightDrop": (0.65,  0.25),
   "Drop":       (0.55,  0.15) ,
    "Panic":      (0.5,  0.10)
}

GOC5_SCN2 = { 
    "Constant" :  (0.88,  0.50)
    , "WorstEver" : (0.63, 0.50)
#    , "Panic" : (0.5, 0.10)
}


FRESET_MODELS = [ ("C", 30*6), ("C", 30*11), ("M", 0) ]
    
FRESET_MODELS_TEST = [ ("C", 30*11),("C", 30*12*3) ]

# Profit in two years
maturity_date = datetime.datetime(2023,2,28)

baz = create_freset_scenarios_per_market_spreads(foo, maturity_date, 
                                                 CURRENT_GOC5_PERCENT, 
                                                 GOC5_SCN2,FRESET_MODELS_TEST)
# baz.sort_values(by="Ticker")
ranked = do_the_ranking_freset(baz, GOC5_SCN2, 20)
ranked.head(50)

,Ticker,TotalRankSum,AvgExpectedYTM
42,CF.PR.A,4,0.086754
72,ENB.PR.F,4,0.083187
1,AIM.PR.C,4,0.082724
73,ENB.PR.H,4,0.082616
71,ENB.PR.D,4,0.081994
38,BPO.PR.R,3,0.080503
9,AZP.PR.B,3,0.080274
36,BPO.PR.N,2,0.079439
50,CPX.PR.E,3,0.079380
76,ENB.PR.P,3,0.079168


In [35]:
ranked = ranked.merge(foo[['Ticker','Rating','MSpread','AvgSpread','ResetDT']],how='inner',on='Ticker')


In [36]:
ranked.sort_values(by=['TotalRankSum','AvgExpectedYTM'],ascending=False).head(20)

,Ticker,TotalRankSum,AvgExpectedYTM,Rating,MSpread,AvgSpread,ResetDT
0,BAM.PR.G,2,5.617911,P2L,3.9479,5.462943,2021-11-15
1,CF.PR.A,2,4.488834,P3L,6.4780,6.125508,2021-09-15
5,BPO.PR.R,2,4.260658,P3I,5.8827,5.790222,2021-09-15
8,BAM.PR.T,2,4.228587,P2L,7.0649,5.462943,2022-03-15
9,ENB.PR.F,2,4.213038,P3H,7.0369,5.946266,2023-06-15
10,AIM.PR.C,2,4.189581,ZR,6.9947,6.243275,2024-03-15
12,ENB.PR.H,2,4.184085,P3H,6.9848,5.946266,2023-09-15
14,ENB.PR.D,2,4.152583,P3H,6.9281,5.946266,2023-03-15
2,HSE.PR.G,1,4.471168,P2L,6.2806,5.462943,2020-06-15
3,HSE.PR.E,1,4.459729,P2L,6.1513,5.462943,2020-03-15


In [10]:
trpg = {'div': .95, 'spread': 296, 'rdate': datetime.datetime(2020,11,15), 'price': 19.36, 'cycle': [2,5,8,11]}

In [53]:
hseg = {'div': 1.15, 'spread': 352, 'rdate': datetime.datetime(2020,6,15), 'price': 16.06, 'cycle': [3,6,9,12]}
bamg = {'div': 0.688, 'spread': 230, 'rdate': datetime.datetime(2021,11,15), 'price': 14.24, 'cycle': [1,4,7,10]}
bamt = {'div': 1.125, 'spread': 231, 'rdate': datetime.datetime(2022,3,15), 'price': 14.16, 'cycle': [1,4,7,10]}

In [57]:
# debug_ytm(bamg, 6*30, 0.88, 0.88, 0, datetime.datetime(2022,3,1))
debug_ytm(bamt, 6*30, 0.88, 0.88, 0, datetime.datetime(2022,6,1))
debug_ytm(bamt, 36*30, 0.88, 0.88, 0, datetime.datetime(2022,6,1))

Using Current Market Spread; far away
MSpread:  7.064915254237288
[(datetime.datetime(2020, 3, 5, 19, 25, 13, 834565), -14.16),
 (datetime.datetime(2020, 4, 30, 0, 0), 0.28125),
 (datetime.datetime(2020, 7, 31, 0, 0), 0.28125),
 (datetime.datetime(2020, 10, 31, 0, 0), 0.28125),
 (datetime.datetime(2021, 1, 31, 0, 0), 0.28125),
 (datetime.datetime(2021, 4, 30, 0, 0), 0.28125),
 (datetime.datetime(2021, 7, 31, 0, 0), 0.28125),
 (datetime.datetime(2021, 10, 31, 0, 0), 0.28125),
 (datetime.datetime(2022, 1, 31, 0, 0), 0.28125),
 (datetime.datetime(2022, 4, 30, 0, 0), 0.199375),
 (datetime.datetime(2022, 6, 1, 0, 0), 10.037866666666668)]
Future Div (int):  0.7975
Maturity Price :  10.037866666666668
Maturity Date:  2022-06-01 00:00:00
Reset  Date:  2022-03-15 00:00:00
Result:  -0.060126356533446716
MSpread:  4.752062146892655
[(datetime.datetime(2020, 3, 5, 19, 25, 13, 836517), -14.16),
 (datetime.datetime(2020, 4, 30, 0, 0), 0.28125),
 (datetime.datetime(2020, 7, 31, 0, 0), 0.28125),
 (dat

0.08033261282349416

3.9514606741573033

In [50]:
def debug_ytm(pref, cliff_days, current_goc5_percent, future_goc5,mspread_offset, mat_date):
    
    mspread_percent = market_spread_from_dividend_in_percent(pref['price'],pref['div'],current_goc5_percent)
    mspread_percent += mspread_offset

    result = compute_ytm_cliff(cliff_days,
                              datetime.datetime.today(), # cur_date
                            pref['price'],
                              pref['div'], # curdiv
                              pref['rdate'], # reset date\
                              pref['spread'],  # irspread_bips
                              mspread_percent, # mspread
                            future_goc5, #future,
                            mat_date, #maturity date
                               pref['cycle'],
                               current_goc5_percent,
                              True
                              )
    print("Result: ", result)
    return result

# debug_ytm(hseg, 180, 0.88, 0.88, 5.87, datetime.datetime(2022,3,1))

In [51]:
def compute_ytm(cur_date, curprice, curdiv, reset_date,
                  ir_spread_bips, 
                  mspread_percent, maturity_goc5_percent, 
                future_div,
                maturity_date, month_cycle, verbose=False) :

    if maturity_date < reset_date:
        maturity_price = curprice
    else:
        maturity_price = share_price_given_ref_rate_and_market_spread(
            maturity_goc5_percent, mspread_percent, ir_spread_bips)
    
    
    flows = build_cashflow_list(cur_date, curprice, 
                            reset_date, 
                            maturity_date,
                            maturity_price,
                            month_cycle,
                            curdiv, 
                            future_div)
    if verbose:
        pp.pprint(flows)
        print("Future Div (int): ", future_div)
        print("Maturity Price : ", maturity_price)
        print("Maturity Date: ", maturity_date)
        print("Reset  Date: ", reset_date)
    
    return xirr(flows, 0.2)

